In [25]:
import pandas as pd
# import our functions
import api_defillama as dfl 
import api_growthe as gtp 
import api_l2beat as l2b

### Run Growthepie

In [26]:
print('getting growthepie fundamentals data')
gtp_act = gtp.get_growthepie_fundamentals()

print('getting growthepie metadata')
gtp_md = gtp.get_growthepie_metadata()

print('yay')

getting growthepie fundamentals data
getting growthepie metadata


In [27]:
gtp_act[gtp_act['date']>='2024-10-01'].sample(5)

metric_key,index,origin_key,date,costs_blobs_eth,costs_blobs_usd,costs_l1_eth,costs_l1_usd,costs_total_eth,costs_total_usd,daa,...,profit_usd,rent_paid_eth,rent_paid_usd,stables_mcap,stables_mcap_eth,tvl,tvl_eth,txcosts_median_eth,txcosts_median_usd,txcount
13589,17451,taiko,2024-10-03,3.531080e-10,8.351819e-07,6.187781,14635.532562,6.187781,14635.532563,182153.0,...,-2345.228090,6.187781,14635.532563,4.292223e+06,1.814716e+03,9.894726e+07,41834.077495,1.681404e-06,0.003977,2492110.0
13448,17205,starknet,2024-10-26,2.306867e-11,5.587448e-08,0.107377,260.077575,0.107377,260.077575,6596.0,...,189.895151,0.107377,260.077575,8.087935e+07,3.339233e+04,6.113607e+08,252410.058472,1.241055e-06,0.003006,52023.0
3255,4853,ethereum,2024-10-06,NaN,NaN,NaN,NaN,NaN,NaN,298946.0,...,NaN,NaN,NaN,8.082169e+10,3.346094e+07,NaN,NaN,4.687916e-04,1.132321,1069768.0
15130,19929,zora,2024-10-28,8.551372e-07,2.142549e-03,0.029332,73.492523,0.029333,73.494665,29412.0,...,998.131717,0.029333,73.494665,2.862396e+05,1.142444e+02,1.946647e+07,7769.487401,2.142570e-06,0.005368,116056.0
6424,8933,manta,2024-10-29,2.600168e-03,6.668948e+00,0.128476,329.516638,0.131076,336.185587,50986.0,...,-7.806234,0.128476,329.516638,1.487324e+07,5.798956e+03,5.113231e+08,199360.714454,2.106994e-08,0.000054,232523.0


In [28]:
gtp_md.head(5)

,name,url_key,chain_type,caip2,evm_chain_id,deployment,chain_name,description,da_layer,symbol,...,l2beat_id,raas,stack,website,twitter,block_explorer,block_explorers,rhino_listed,rhino_naming,origin_key
0,Ethereum,ethereum,L1,eip155-1,1.0,PROD,Ethereum,"Ethereum, proposed by Vitalik Buterin in 2013 ...",None,ETH,...,ethereum,Self-hosted,"{'label': 'Custom', 'url': None}",https://ethereum.org/,https://twitter.com/ethereum,https://etherscan.io/,"{'Etherscan': 'https://etherscan.io/', 'Blocks...",True,ETHEREUM,ethereum
1,All L2s,all-l2s,-,None,NaN,PROD,-,None,None,None,...,None,None,None,None,None,None,None,False,None,all_l2s
2,Arbitrum One,arbitrum,L2,eip155-42161,42161.0,PROD,Arbitrum,Arbitrum One is developed by Offchain Labs. It...,Ethereum (blobs),ARB,...,arbitrum,Self-hosted,"{'label': 'Arbitrum Nitro', 'url': 'https://ar...",https://arbitrum.io/,https://twitter.com/arbitrum,https://arbiscan.io/,"{'Arbiscan': 'https://arbiscan.io/', 'Blocksco...",True,ARBITRUM,arbitrum
3,Base,base,L2,eip155-8453,8453.0,PROD,Base,"Base is an Ethereum Layer 2 offering a secure,...",Ethereum (blobs),None,...,base,Self-hosted,"{'label': 'OP Stack, Superchain', 'url': 'http...",https://base.org/,https://twitter.com/base,https://basescan.org/,"{'BaseScan': 'https://basescan.org/', 'Blocksc...",True,BASE,base
4,Blast,blast,L2,eip155-81457,81457.0,PROD,Blast,Blast is an EVM-compatible Optimistic Rollup w...,Ethereum (blobs),BLAST,...,blast,Self-hosted,"{'label': 'OP Stack', 'url': 'https://docs.opt...",https://blast.io/en,https://twitter.com/Blast_L2,https://blastscan.io/,{'Blast Explorer': 'https://blastscan.io/'},True,BLAST,blast


In [29]:
num_chains = len(gtp_md[gtp_md['chain_name']!= '-'])
print(f"Growthepie: Total Amount of Chains: {num_chains}")

Growthepie: Total Amount of Chains: 24
